In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from utils.classifications_utils import *
from utils.data_processing_utils_lin_edit import *
from utils.data_visualization_utils import *
from utils.metrics_utils import *
from utils.grid_search_utils import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy import signal
from utils.reduce_noise_utils import *
import pickle
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Drone 4 Data

In [2]:
level_0_wind = load_data(0, 6, "drone1")
level_0_wind = separate_data_based_on_apparatus(level_0_wind)

level_1_wind = load_data(1, 6, "drone1")
level_1_wind = separate_data_based_on_apparatus(level_1_wind)

level_2_wind = load_data(2, 6, "drone1")
level_2_wind = separate_data_based_on_apparatus(level_2_wind)


In [3]:
acc_no_wind, gyro_no_wind, _, _ = level_0_wind.values()
label_0 = [0 for x in range(acc_no_wind.shape[0])]

acc_level_1_wind, gyro_level_1_wind, _, _ = level_1_wind.values()
label_1 = [1 for x in range(acc_level_1_wind.shape[0])]

acc_level_2_wind, gyro_level_2_wind, _, _ = level_2_wind.values()
label_2 = [2 for x in range(acc_level_2_wind.shape[0])]

# Reduce noise

In [4]:
gyro_no_wind_reduced = reduce_noise_with_FFT(gyro_no_wind, [100, 300, 200])# Reduce noise

In [5]:
gyro_level_1_reduced = reduce_noise_with_FFT(gyro_level_1_wind, [2500, 5000, 2000])

In [6]:
gyro_level_2_reduced = reduce_noise_with_FFT(gyro_level_2_wind, [6000, 2000, 1000])

# Split No Wind Data

In [7]:
X_train_acc_0, X_test_acc_0, y_train_acc_0, y_test_acc_0 = \
train_test_split(acc_no_wind, label_0, test_size=0.2, shuffle=False)

X_train_gyro_0, X_test_gyro_0, y_train_gyro_0, y_test_gyro_0 = \
train_test_split(gyro_no_wind_reduced, label_0, test_size=0.2, shuffle=False)

# Split Level 1 Wind

In [8]:
X_train_acc_1, X_test_acc_1, y_train_acc_1, y_test_acc_1 = \
train_test_split(acc_level_1_wind, label_1, test_size=0.2, shuffle=False)

X_train_gyro_1, X_test_gyro_1, y_train_gyro_1, y_test_gyro_1 = \
train_test_split(gyro_level_1_reduced, label_1, test_size=0.2, shuffle=False)

# Split Level 2 Wind

In [9]:
X_train_acc_2, X_test_acc_2, y_train_acc_2, y_test_acc_2 = \
train_test_split(acc_level_2_wind, label_2, test_size=0.2, shuffle=False)

X_train_gyro_2, X_test_gyro_2, y_train_gyro_2, y_test_gyro_2 = \
train_test_split(gyro_level_2_reduced, label_2, test_size=0.2, shuffle=False)

# Stack data up

# Gyro

In [10]:
X_train_gyro = X_train_gyro_0.append(X_train_gyro_1)
X_train_gyro = X_train_gyro.append(X_train_gyro_2)
y_train_gyro = np.hstack((y_train_gyro_0, y_train_gyro_1, y_train_gyro_2))
display(X_train_gyro.shape)

(86400, 3)

In [11]:
X_test_gyro = X_test_gyro_0.append(X_test_gyro_1)
X_test_gyro = X_test_gyro.append(X_test_gyro_2)
y_test_gyro = np.hstack((y_test_gyro_0, y_test_gyro_1, y_test_gyro_2))

# Accelerometer

In [12]:
X_train_acc = X_train_acc_0.append(X_train_acc_1)
X_train_acc = X_train_acc.append(X_train_acc_2)

y_train_acc = np.hstack((y_train_acc_0, y_train_acc_1, y_train_acc_2))

In [13]:
X_test_acc = X_test_acc_0.append(X_test_acc_1)
X_test_acc = X_test_acc.append(X_test_acc_2)

y_test_acc = np.hstack((y_test_acc_0, y_test_acc_1, y_test_acc_2))

# Transform and generate features

In [14]:
from transformers.featureGenerator import FeatureGenerator

# Train a random forest classifier and evaluate 

In [15]:
from sklearn.ensemble import RandomForestClassifier

# Classify 0, min and max levels of airflows

# Gyro

# Stack up no wind, level 1 wind and level 2 wind data 

In [16]:
X_train_gyro = X_train_gyro_0.append(X_train_gyro_1)
X_train_gyro = X_train_gyro.append(X_train_gyro_2)

y_train_gyro = np.hstack((y_train_gyro_0, y_train_gyro_1, y_train_gyro_2))

In [17]:
X_test_gyro = X_test_gyro_0.append(X_test_gyro_1)
X_test_gyro = X_test_gyro.append(X_test_gyro_2)

y_test_gyro = np.hstack((y_test_gyro_0, y_test_gyro_1, y_test_gyro_2))

In [18]:
feature_generator_gyro2 = FeatureGenerator(1)
feature_generator_gyro2.fit(X_train_gyro, 3)

FeatureGenerator(sliding_window=1)

In [19]:
X_train_gyro = feature_generator_gyro2.transform(X_train_gyro, 'gyro')
y_train_gyro = reduce_label_amount(y_train_gyro, 3)

28701


In [20]:
feature_generator_gyro_test2 = FeatureGenerator(1)
feature_generator_gyro_test2.fit(X_test_gyro, 3)

FeatureGenerator(sliding_window=1)

In [21]:
X_test_gyro = feature_generator_gyro_test2.transform(X_test_gyro, 'gyro')
y_test_gyro = reduce_label_amount(y_test_gyro, 3)

7101


# run pickled random forest classifier and predict

In [22]:
pickled_clf = pickle.load(open("randomforest_gyro2.sav", 'rb'))
y_pred = pickled_clf.predict(X_test_gyro)
pickled_clf.score(X_test_gyro, y_test_gyro)

0.9964324273576491

# Confusion Matrix: Gyro

In [23]:
get_confusion_matrix(y_test_gyro, y_pred)

,actual_0,actual_1,actual_2
predicted_0,7088,8,5
predicted_1,24,7071,6
predicted_2,0,33,7068
